In [ ]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# ---------------------------
# 1. 生成模拟数据
# ---------------------------
np.random.seed(42)
X = 2 * np.random.rand(100, 3)          # 三个特征
y = 4 + 3*X[:,0] + 0.5*X[:,1] + np.random.randn(100)  # y = 4 + 3x1 + 0.5x2 + noise

# ---------------------------
# 2. 标准化（梯度下降推荐）
# ---------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 在 X 前加一列 1 对应截距项
X_b = np.c_[np.ones((X_scaled.shape[0], 1)), X_scaled]  # (100,4)

# ---------------------------
# 3. 带 L2 正则化的梯度下降实现
# ---------------------------
def ridge_gradient_descent(X, y, alpha=0.1, lambda_=1.0, n_iterations=1000):
    """
    X: 特征矩阵 (带截距项)
    y: 目标向量
    alpha: 学习率
    lambda_: L2 正则化强度
    n_iterations: 迭代次数
    """
    m, n = X.shape#m 是样本数，n 是特征数（如果 X 已经加了截距列，n = 特征数 + 1）
    theta = np.zeros(n)  # 初始化参数,theta 初始化为 0 向量，是每个特征的回归系数

    for iteration in range(n_iterations):
        y_pred = X.dot(theta)#y_pred 是当前模型预测值
        error = y_pred - y#error 是预测误差（残差）
        # 梯度：普通梯度 + 正则化梯度（截距项不正则化）
        gradients = (2/m) * (X.T.dot(error) + lambda_ * np.r_[0, theta[1:]])#(2/m) * X.T.dot(error) 是 普通最小二乘梯度,
        #lambda_ * np.r_[0, theta[1:]] 是 L2 正则化梯度,np.r_[0, theta[1:]] 表示截距项不正则化,两者相加就是带 L2 正则化的梯度
        theta -= alpha * gradients#学习率 alpha 控制每次迭代的步长,参数更新公式：theta = theta - η * gradient

        if iteration % 100 == 0:
            mse = mean_squared_error(y, X.dot(theta))
            print(f"Iteration {iteration}: MSE={mse:.4f}")#每 100 次迭代打印一次 MSE，用于观察收敛情况

    return theta

# ---------------------------
# 4. 使用梯度下降训练
# ---------------------------
theta_gd = ridge_gradient_descent(X_b, y, alpha=0.1, lambda_=1.0, n_iterations=1000)
print("\nGradient Descent Coefficients:", theta_gd)
print("Gradient Descent MSE:", mean_squared_error(y, X_b.dot(theta_gd)))

# ---------------------------
# 5. 使用 sklearn Ridge 对比
# ---------------------------
ridge = Ridge(alpha=1.0, fit_intercept=True)
ridge.fit(X_scaled, y)  # sklearn 自动处理截距项
print("\nSklearn Ridge Coefficients:", np.r_[ridge.intercept_, ridge.coef_])
print("Sklearn Ridge MSE:", mean_squared_error(y, ridge.predict(X_scaled)))


Iteration 0: MSE=38.4702
Iteration 100: MSE=0.9207
Iteration 200: MSE=0.9207
Iteration 300: MSE=0.9207
Iteration 400: MSE=0.9207
Iteration 500: MSE=0.9207
Iteration 600: MSE=0.9207
Iteration 700: MSE=0.9207
Iteration 800: MSE=0.9207
Iteration 900: MSE=0.9207

Gradient Descent Coefficients: [7.44338454 1.76835891 0.25829139 0.16029311]
Gradient Descent MSE: 0.9207245432708794

Sklearn Ridge Coefficients: [7.44338454 1.76835891 0.25829139 0.16029311]
Sklearn Ridge MSE: 0.9207245432708794


总结

梯度下降可以成功学习带 L2 正则化的线性回归系数

对比 Sklearn Ridge，结果一致 → 模型正确

MSE 收敛到 0.92，说明拟合效果不错

梯度下降优点：可扩展到大数据或自定义正则化

Sklearn Ridge 优点：解析解直接求得，速度快、稳定